In [28]:
import pandas as pd
import numpy as np


raw_dataset=pd.read_excel('PSP_Jan_Feb_2019.xlsx')
raw_dataset=pd.DataFrame(raw_dataset)
raw_dataset.tail()

,Unnamed: 0,tmsp,country,amount,success,PSP,3D_secured,card
50405,50405,2019-02-28 23:45:39,Switzerland,415,0,UK_Card,0,Visa
50406,50406,2019-02-28 23:46:48,Austria,91,0,UK_Card,0,Master
50407,50407,2019-02-28 23:47:04,Austria,91,0,UK_Card,0,Master
50408,50408,2019-02-28 23:47:36,Austria,91,0,UK_Card,0,Master
50409,50409,2019-02-28 23:48:19,Austria,91,1,Moneycard,0,Master


In [42]:
raw_dataset.card.unique()

array(['Visa', 'Diners', 'Master'], dtype=object)

,Is_Germany,Is_Austria,Is_Switzerland
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,1.0,0.0


In [36]:
print(raw_dataset.loc[0,'country'])


Germany


In [38]:
prepros_dataset=pd.DataFrame()
for i in range(len(raw_dataset)):
    if  raw_dataset.iloc[i,2]=='Germany':
        prepros_dataset.loc[i,'Is_Germany']=1
        prepros_dataset.loc[i,'Is_Austria']=0
        prepros_dataset.loc[i,'Is_Switzerland']=0
    elif raw_dataset.iloc[i,2]=='Austria':
        prepros_dataset.loc[i,'Is_Germany']=0
        prepros_dataset.loc[i,'Is_Austria']=1
        prepros_dataset.loc[i,'Is_Switzerland']=0
    elif raw_dataset.iloc[i,2]=='Switzerland':
        prepros_dataset.loc[i,'Is_Germany']=0
        prepros_dataset.loc[i,'Is_Austria']=0
        prepros_dataset.loc[i,'Is_Switzerland']=1

prepros_dataset.head()

,Is_Germany,Is_Austria,Is_Switzerland
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,1.0,0.0


In [39]:
prepros_dataset.tail()

,Is_Germany,Is_Austria,Is_Switzerland
50405,0.0,0.0,1.0
50406,0.0,1.0,0.0
50407,0.0,1.0,0.0
50408,0.0,1.0,0.0
50409,0.0,1.0,0.0


In [41]:
for i in range(len(raw_dataset)):
    if  raw_dataset.iloc[i,5]=='UK_Card':
        prepros_dataset.loc[i,'Is_UK_Card']=1
        prepros_dataset.loc[i,'Is_Simplecard']=0
        prepros_dataset.loc[i,'Is_Moneycard']=0
        prepros_dataset.loc[i,'Is_Goldcard']=0
    elif raw_dataset.iloc[i,5]=='Simplecard':
        prepros_dataset.loc[i,'Is_UK_Card']=0
        prepros_dataset.loc[i,'Is_Simplecard']=1
        prepros_dataset.loc[i,'Is_Moneycard']=0
        prepros_dataset.loc[i,'Is_Goldcard']=0
    elif raw_dataset.iloc[i,5]=='Moneycard':
        prepros_dataset.loc[i,'Is_UK_Card']=0
        prepros_dataset.loc[i,'Is_Simplecard']=0
        prepros_dataset.loc[i,'Is_Moneycard']=1
        prepros_dataset.loc[i,'Is_Goldcard']=0
    elif raw_dataset.iloc[i,5]=='Goldcard':
        prepros_dataset.loc[i,'Is_UK_Card']=0
        prepros_dataset.loc[i,'Is_Simplecard']=0
        prepros_dataset.loc[i,'Is_Moneycard']=0
        prepros_dataset.loc[i,'Is_Goldcard']=1

prepros_dataset.head()

,Is_Germany,Is_Austria,Is_Switzerland,Is_UK_Card,Is_Simplecard,Is_Moneycard,Is_Goldcard
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [45]:
for i in range(len(raw_dataset)):
    if  raw_dataset.iloc[i,7]=='Visa':
        prepros_dataset.loc[i,'Is_Visa']=1
        prepros_dataset.loc[i,'Is_Diners']=0
        prepros_dataset.loc[i,'Is_Master']=0
    elif raw_dataset.iloc[i,7]=='Diners':
        prepros_dataset.loc[i,'Is_Visa']=0
        prepros_dataset.loc[i,'Is_Diners']=1
        prepros_dataset.loc[i,'Is_Master']=0
    elif raw_dataset.iloc[i,7]=='Master':
        prepros_dataset.loc[i,'Is_Visa']=0
        prepros_dataset.loc[i,'Is_Diners']=0
        prepros_dataset.loc[i,'Is_Master']=1

prepros_dataset.head()

,Is_Germany,Is_Austria,Is_Switzerland,Is_UK_Card,Is_Simplecard,Is_Moneycard,Is_Goldcard,Is_Visa,Is_Diners,Is_Master,Master
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,NaN


In [46]:
prepros_dataset.drop("Master",axis=1,inplace=True)
prepros_dataset.head()

,Is_Germany,Is_Austria,Is_Switzerland,Is_UK_Card,Is_Simplecard,Is_Moneycard,Is_Goldcard,Is_Visa,Is_Diners,Is_Master
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
